In [ ]:
from selenium.webdriver import Chrome
import pandas as pd
import time
import os

url = "https://www.cathaybk.com.tw/cathaybk/personal/credit-card/cards/intro/list/"

# 開啟並取得國泰信用卡網頁。
driver = Chrome("./chromedriver.exe")
# 注意是否要放大，或自訂螢幕大小。
driver.maximize_window()
time.sleep(1)
driver.get(url)

In [ ]:
#每個迴圈都要停一下 確定指令運作已完成,不然有時候跑太快 頁面還沒捲下去會找不到id
#確定你要的id有在畫面上,不然原始碼可能不會載入


#抓取每一個分頁的卡片名稱
title = []
for page in range(4):
    print("page=",page)
    time.sleep(1)
    js="var action=document.documentElement.scrollTop=10000"
    time.sleep(1)
    driver.execute_script(js)
    time.sleep(1)
    driver.find_element_by_id("layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_"+str(page)).click()
    time.sleep(1)
    print("我在第",page+1,"頁")
    ps = driver.find_elements_by_class_name("card-name")
    for p in ps:
        name = p.text
        title.append(name)
    
#回到第一頁
driver.find_element_by_id("layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_0").click()

In [ ]:
#整理list內容,把為空的部分去除掉
#並且把字之間的空白去掉
title = [x for x in title if x != '']
title2 = []
for y in title:
    y = y.replace(" ", "")
    y = y.replace("(COMBO)", "")
    title2.append(y)
title2 = title2[:23]
print(title2)
print(len(title2))

In [ ]:
#滾輪往下滾
#滾輪的scrollTop=X是絕對位子
def scrolltop(pos):
    js = "document.documentElement.scrollTop=%s" % pos
    driver.execute_script(js)
    
#滾輪滾到底
def scrolldown():
    js = "document.documentElement.scrollTop=10000"
    driver.execute_script(js)
    
#去到下一頁
def backpage(page):
    driver.find_element_by_id(
        "layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_{}".format(page-1)).click()
    # 給他五秒鐘的時間跑
    time.sleep(5)
    
#抓取網頁內的資料
context = []
title3 = []
temp = []
times = 0
def catch():
    
##############################
#                            #
#下面是資料依據上標分開放的版本 #
#                            #
##############################
    
    global times
    #先判斷網頁有幾個上標
    title = []

    for num in range(10):
        try:
            lenth = driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(num+1)).text
            #確保串列內字元不為空
            if lenth != "":
                title.append(lenth)
            else:
                continue
        except:
            break
    # 根據上標數做迴圈
    for item in range(len(title)):
        try:
            driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(item + 1)).click()
            data = driver.find_element_by_id("layout_0_rightcontent_1_tab0"+str(item+1)+"_0_divParagraph")
            x = data.text
            y = title[item]+":"+x
            title3.append(title2[times])
            #把每個上標的內容先存到temp裡面
            temp.append(y)
        except:
            #這裡要新增卡友權益
            continue
    times += 1
    # 上一頁到卡片清單頁
    for item2 in range(len(title)):
        driver.back()
        
        
##############################
#                            #
#下面是所有資料集合在一起的版本 #
#                            #
##############################


#     temp = []
#     #先判斷網頁有幾個上標
#     title = []
#     for num in range(10):
#         try:
#             lenth = driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(num+1)).text
#             #確保串列內字元不為空
#             if lenth != "":
#                 title.append(lenth)
#             else:
#                 continue
#         except:
#             break
#     # 根據上標數做迴圈
#     for item in range(len(title)):
#         try:
#             driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(item + 1)).click()
#             data = driver.find_element_by_id("layout_0_rightcontent_1_tab0"+str(item+1)+"_0_divParagraph")
#             x = data.text
            
#             #把每個上標的內容先存到temp裡面
#             temp.append(x)
#         except:
#             #這裡要新增卡友權益
#             continue
#     sstr = ""
#     sstr = sstr.join(temp)
#     sstr = sstr.replace(" ","")
#     sstr = sstr.replace("\r","")
#     sstr = sstr.replace("\n","")
#     context.append(sstr)
#     # 上一頁到卡片清單頁
#     for item2 in range(len(title)):
#         driver.back()

In [ ]:
# 起始位置設為200，注意螢幕大小，我的螢幕為14吋。
page = 1
while page < 5:
    try:
        pos = 200
        for i in range(10):
            pos += 220
            scrolltop(pos)
            # 注意抓取的值為list，要一個一個取出。
            driver.find_elements_by_xpath("//div[@class='card-features']/a[@class='link-learn-more']")[i].click()
            catch()
            print("現在到第幾頁:", page, "到第幾張卡片:", i + 1)
            time.sleep(1)
            if page == 1:
                driver.back()
                scrolltop(pos)
                time.sleep(1)
            else:
                driver.back()
                scrolldown()
                backpage(page)
        page += 1
        backpage(page)
    except:
        break
print("好像沒東西喔!!!")


In [ ]:
#去除不必要的跳脫字元
for newtext in temp:
    newtext = newtext.replace(" ","")
    newtext = newtext.replace("\r","")
    newtext = newtext.replace("\n","")
    context.append(newtext)
print(context)

In [ ]:
#抓蝦皮的那一頁

from bs4 import BeautifulSoup
import requests

text = []
p = requests.get(
        "https://www.cathaybk.com.tw/cathaybk/promo/event/credit-card/product/shopee/index.html#info"
)
p.encoding = "utf-8"
x = BeautifulSoup(p.text, "html.parser")

#取最上面那區塊
a1 = x.find("div",{
    "class" : "container top"
})
a2 = a1.find("div",{
        "class" : "left"
    }).text
a2 = a2.replace("\n", "")
text.append(a2)

#取第二區塊
a3 = x.find("div",{
        "class" : "container bottom"
    })
a4 = a3.find("div",{
        "class" : "item-container"
}).text

a4 = a4.replace("\n", "")
text.append(a4)

#取第三區塊
a5 = x.find("section",{
        "class" : "section-fresh"
    }).text

a5 = a5.replace("\n", "")
text.append(a5)

#取第四區塊
a6 = x.find("section",{
    "class" : "section-binding"
}).text
a6 = a6.replace("\n", "")
text.append(a6)

#取第五區塊
a7 = x.find("section",{
    "class" : "section-discount"
}).text
a7 = a7.replace("\n", "")
text.append(a7)


In [ ]:
#把蝦皮的資料跟原本的主資料結合
for j in range(len(text)):
    title3.insert(6, title2[1])

for i in range(len(text)):
    context.insert(6, text[-(i+1)])

In [ ]:
result = pd.DataFrame({
    "Title":title3,
    "context":context
})
result.to_csv("cathaybk.csv", encoding = "utf-8", index = False)
result